In [ ]:
# Suppress warnings
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

# Direct access
from pprint import pprint
import s3fs
import earthaccess as ea
import boto3
import fsspec

# datetime
import datetime as dt

# api to read files and data process
import gzip
import xarray as xr
import h5py
import numpy as np
import pandas as pd

In [ ]:
# if you want to print the structure of a HDF5 file you can use the following code:
def h5printR(item, leading = ''):
    for key in item:
        if isinstance(item[key], h5py.Dataset):
            print(leading + key + ': ' + str(item[key].shape))
        else:
            print(leading + key)
            h5printR(item[key], leading + '  ')

In [ ]:
auth = ea.login(strategy="netrc", persist=True) # use 'interactive' strategy the first time to log in

In [ ]:
#### dataset2 in ASCII format with compression ####

# step1: provdie the daac name and short name of dataset
    # define the name of daac and dataset
DAAC2 = 'GHRC_DAAC'
ssh_short_name2 = "gpmmrms" # short name of dataset. 

# step2: get links #
results2 = ea.search_data(
    short_name=ssh_short_name2,
    cloud_hosted=True,
    temporal=("2015-07-01", "2015-07-02"),
)

    # data_link list
https_links2 = []  # external link, todo: find ways to directly access data using external link
s3_links2 = []  # s3 link to directly accessing  data in AWS

for granule in results2:
    https_links2.extend(granule.data_links(access="on_prem"))
    s3_links2.extend(granule.data_links(access="direct"))
    
# step3: select proper links for use (similar part to what we did when we worked in local enviroment)

# step4: get credentials and open files with credentials #
s3_fs2 = ea.get_fsspec_https_session() # open files with https links, works both in and out AWS 
f2 = s3_fs2.open(https_links2[0], mode='rb')  # longer credential for 90 days

#### read data under zip file (.gz)####

# step5: open zip file #
data_ori = gzip.open(f2, 'rb')

# step6: read data # 
datainfo = pd.read_csv(data_ori,header = None)[:6]  # read file information in the first 6 rows,which can be ignored
ascii_grid = np.loadtxt(data_ori,skiprows=6)  # read interested dataset 